## Load data and packages

In [1]:
import random
import numpy as np
import scanpy as sc
import anndata as an
import scipy.stats as st
from scipy.spatial import KDTree
import math
from numba import njit
from numba import jit

In [2]:
def LoadData(path):
    adata = sc.read_visium(path)
    adata.var_names_make_unique()
    return adata

In [3]:
path ="/Users/juliafoyer/Documents/Skolarbete/Masters_thesis/human_breast_cancer_ST_data"
adata = LoadData(path)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [4]:
X = np.array(adata.X.todense())

In [5]:
adata.X = X

In [6]:
X.shape

(3798, 36601)

In [7]:
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)

In [8]:
adata = adata[:,adata.var.highly_variable.values]

In [9]:
# Another option is to save genes, reload data and then filter
adata.X = (np.exp(adata.X) -1).round(0)

In [10]:
X = adata.X

In [11]:
spots = adata.obs_names
genes = adata.var_names
n_spots = len(adata.obs_names)
n_genes = len(adata.var_names)

## Assign random factors

In [12]:
def assign_random(adata, K):
    umi_factor_list = []
    all_n_umis = adata.X.sum(axis=1)
    for n_UMIs in all_n_umis:
        factors = np.random.randint(low = 0,
                                    high = K,
                                    size = int(n_UMIs))
        umi_factor_list.append(factors)
    return umi_factor_list

In [13]:
umi_factors = assign_random(adata, 3)

## Get IDs and count matrices

In [14]:
def get_ids_dt_wt(adata, umi_factors, K):
    n_spots, n_genes = adata.X.shape
    X = adata.X
    ids = []
    dt = np.zeros((n_spots, K))
    wt = np.zeros((K, n_genes))
    for spot in range(n_spots):
        ids_spot = []
        spot_list = umi_factors[spot].tolist()
        start = 0
        end = 0
        for gene in range(n_genes):
            n_umis = int(X[spot, gene])
            ids_spot += [gene] * n_umis
            end += n_umis
            for factor in range(K):
                wt[factor, gene] += spot_list[start: end].count(factor)
            start = end
        ids.append(ids_spot)
        for factor in range(K):
            dt[spot, factor] = spot_list.count(factor)
    return ids, dt, wt

In [15]:
ids, dt, wt = get_ids_dt_wt(adata, umi_factors, 3)

In [16]:
assert(np.all(np.sum(dt,axis=1) == np.sum(X,axis=1))), "total UMI's in spot and dt rowsums must be equal"

## Functions for calculating theta and phi

In [17]:
def get_theta(dt):
    n_spots, K = dt.shape
    theta = np.zeros((n_spots, K))
    for spot in range(n_spots):
        for factor in range(K):
            theta[spot, factor] = dt[spot, factor]/np.sum(dt, axis=1)[spot]
    return theta

In [18]:
theta = get_theta(dt)

In [19]:
def get_phi(wt): # I don't use this...
    phi = wt/wt.sum(axis=1, keepdims=True)
    return phi

In [20]:
phi = get_phi(wt)

## Get neighbours and graph edges

In [21]:
def findKNN(adata, n_neighbours):
    kd = KDTree(adata.obsm["spatial"])
    dist,indx = kd.query(adata.obsm["spatial"], k = n_neighbours + 1)
    return dist, indx

In [22]:
dist, indx = findKNN(adata, 6)

In [23]:
u,c = np.unique(indx[:,-1],return_counts = True) # Can't remember why we did this...

In [24]:
def remove_false_neighbours(dist, indx, eps = 300):
    dist_sel = []
    indx_sel = []
    for i in range(len(dist)):
        dist_sel.append([])
        indx_sel.append([])
        for j in range(len(dist[i])):
            if dist[i][j] < eps:
                dist_sel[i].append(dist[i][j])
                indx_sel[i].append(indx[i][j])
        dist_sel[i] = np.array(dist_sel[i])
        indx_sel[i] = np.array(indx_sel[i])
    return np.array(dist_sel, dtype=object), np.array(indx_sel, dtype=object)

In [25]:
dist_sel, indx_sel = remove_false_neighbours(dist, indx)

In [26]:
def get_E(indx_sel):
    edges = 0
    for i in indx_sel:
        edges += len(i) - 1
    return edges

In [27]:
E = get_E(indx_sel)

## Get first lambda and l_dd

In [28]:
@njit(parallel=False)
def Bhatt_dist(p, q, X):
    BC = 0
    for x in range(X):
        BC += math.sqrt(p[x]*q[x])
    DB = - math.log(BC)
    return DB

In [29]:
lambda0 = np.random.gamma(0.01, 0.01, 1)

In [30]:
lambda0

array([4.92482907e-17])

In [31]:
def first_edge_influence(lambda0, theta, indx_sel, K):
    edge_influence = []
    n_spots = len(indx_sel)
    X = K
    for spot in range(n_spots):
        p = theta[0]
        print(p)
        influences = []
        neighbours = indx_sel[spot][1:]
        for neighbour in neighbours:
            q = theta[neighbour]
            print(q)
            print(Bhatt_dist(p, q, X))
            influences.append(np.random.exponential(lambda0 + Bhatt_dist(p, q, X)))
        edge_influence.append(influences)
    return np.array(edge_influence, dtype=object)

In [32]:
edge_influence = first_edge_influence(lambda0, theta, indx_sel, 3)

[0.34823848 0.33130081 0.3204607 ]
[0.3283458  0.35577722 0.31587697]
0.00037263054073066444
[0.34073014 0.30422334 0.35504653]
0.0007522773990893102
[0.34489938 0.33310201 0.32199861]
6.162599859163077e-06
[0.40853659 0.32926829 0.26219512]
0.0026684361306585887
[0.32150033 0.34829203 0.33020764]
0.00040978321136136606
[0.32842849 0.33725721 0.3343143 ]
0.00023159970334198814
[0.34823848 0.33130081 0.3204607 ]
[0.34760766 0.32464115 0.3277512 ]
3.7547550770668607e-05
[0.33518115 0.34448624 0.32033261]
0.00012671010460679267
[0.34428836 0.32971111 0.32600053]
1.8457352460779942e-05
[0.32919255 0.32897072 0.34183673]
0.0003085244922675066
[0.33175 0.341   0.32725]
0.00015275611090169097
[0.33051742 0.33072862 0.33875396]
0.00024277306064046298
[0.34823848 0.33130081 0.3204607 ]
[0.35625    0.34791667 0.29583333]
0.00037060927965110716
[0.34734513 0.30973451 0.34292035]
0.000371950519826936
[0.35057471 0.3591954  0.29022989]
0.0006583610401921495
[0.32097004 0.34522111 0.33380884]
0.0004

[0.35135135 0.35521236 0.29343629]
0.0005094231225320776
[0.34016393 0.31352459 0.34631148]
0.00039694472113486936
[0.3591954  0.32471264 0.31609195]
6.646781240267153e-05
[0.34169279 0.28526646 0.37304075]
0.0018758109056113178
[0.34823848 0.33130081 0.3204607 ]
[0.33470226 0.33880903 0.32648871]
0.00010215943134012255
[0.32638165 0.33159541 0.34202294]
0.00035267092261421584
[0.33521127 0.30046948 0.36431925]
0.001142094115679742
[0.31343284 0.33378562 0.35278155]
0.0008488516210404993
[0.33478261 0.32753623 0.33768116]
0.00018433680850651015
[0.33933162 0.32990574 0.33076264]
7.032972408193213e-05
[0.34823848 0.33130081 0.3204607 ]
[0.32739761 0.3253625  0.34723989]
0.0004428912033790663
[0.33506261 0.33255814 0.33237925]
0.00011852688757852177
[0.33686195 0.33976218 0.32337587]
7.720794332233505e-05
[0.33468651 0.32972485 0.33558863]
0.00015541018310254043
[0.33148597 0.32906131 0.33945272]
0.00024183811397116082
[0.34241998 0.32465705 0.33292297]
8.851164533960334e-05
[0.34823848 

[0.34433541 0.32292596 0.33273863]
9.000702833929025e-05
[0.33733055 0.34775808 0.31491137]
0.00015524718315026304
[0.35055592 0.32243296 0.32701112]
4.856476000786517e-05
[0.35373537 0.33123312 0.3150315 ]
2.2359199230864818e-05
[0.34823848 0.33130081 0.3204607 ]
[0.33173077 0.33942308 0.32884615]
0.000151881307724026
[0.32538194 0.33878317 0.3358349 ]
0.00030492653381838307
[0.33419689 0.33231276 0.33349034]
0.00013754047270098294
[0.33774834 0.33333333 0.32891832]
6.920207387178634e-05
[0.34484795 0.32388544 0.33126661]
6.992605494614316e-05
[0.32883817 0.33195021 0.33921162]
0.0002724677563519282
[0.34823848 0.33130081 0.3204607 ]
[0.33979136 0.32518629 0.33502235]
0.0001210554612834483
[0.32691763 0.34673082 0.32635154]
0.0002696116175020467
[0.33913841 0.3238619  0.33699969]
0.00015527823479949342
[0.33322991 0.34439963 0.32237046]
0.00014756388224743526
[0.33988764 0.32945425 0.33065811]
6.655765721977446e-05
[0.33618402 0.33588912 0.32792687]
8.24676301467323e-05
[0.34823848 0.

0.0001598747561184952
[0.33141962 0.32620042 0.34237996]
0.00029526068532326427
[0.34065073 0.32418659 0.33516268]
0.00012263696579508526
[0.34764278 0.32234573 0.3300115 ]
6.586284990022973e-05
[0.32349323 0.34112341 0.33538335]
0.00034879895296827985
[0.34823848 0.33130081 0.3204607 ]
[0.34354575 0.32495915 0.3314951 ]
6.997440949894889e-05
[0.32837029 0.32626639 0.34536333]
0.00038852596559091134
[0.32588774 0.33906071 0.33505155]
0.0002890136808659653
[0.33817062 0.34256816 0.31926121]
8.458712801059883e-05
[0.33526358 0.32408147 0.34065495]
0.00023573852436340102
[0.33609959 0.32949132 0.3344091 ]
0.00012936363989332335
[0.34823848 0.33130081 0.3204607 ]
[0.31765613 0.31842714 0.36391673]
0.0011062399423137738
[0.33142857 0.33274725 0.33582418]
0.0001946866909239891
[0.32339657 0.35140018 0.32520325]
0.0003865387657409468
[0.33460699 0.31495633 0.35043668]
0.0005065263125945071
[0.32681683 0.3319167  0.34126647]
0.0003337678676616677
[0.31974374 0.34420501 0.33605125]
0.0004583303

0.000735847077623998
[0.3015873  0.35064935 0.34776335]
0.0012554161553599182
[0.34823848 0.33130081 0.3204607 ]
[0.34125677 0.33490233 0.3238409 ]
2.6975676781010813e-05
[0.33517459 0.34069836 0.32412705]
0.00010051199406088699
[0.33301916 0.33239083 0.33459001]
0.00016167162814469864
[0.32471314 0.33567392 0.33961295]
0.0003518517748147767
[0.36134088 0.31649978 0.32215934]
0.00014617734042079395
[0.32380074 0.32687577 0.34932349]
0.0005409064729018347
[0.34823848 0.33130081 0.3204607 ]
[0.34927697 0.32035595 0.33036707]
8.404810987231157e-05
[0.32201987 0.34354305 0.33443709]
0.00038667521228041466
[0.3447205  0.30952381 0.34575569]
0.0004298090433807372
[0.31058824 0.36411765 0.32529412]
0.0009351980261655794
[0.33892617 0.31431767 0.34675615]
0.00040251852695361815
[0.34823848 0.33130081 0.3204607 ]
[0.3324937  0.34256927 0.32493703]
0.00014593664949679106
[0.31862745 0.33088235 0.3504902 ]
0.0006653293056737193
[0.33793103 0.30344828 0.35862069]
0.0008812960876869683
[0.325 0.325

...

[0.34074074 0.32063492 0.33862434]
0.0001892102204336034
[0.30673317 0.35860349 0.33466334]
0.001005925205694145
[0.34163037 0.30678283 0.35158681]
0.0006122189181486948
[0.34823848 0.33130081 0.3204607 ]
[0.33915575 0.3216885  0.33915575]
0.00019789216698137393
[0.35687168 0.33181473 0.31131359]
5.963876514137236e-05
[0.34074074 0.32063492 0.33862434]
0.0001892102204336034
[0.36107987 0.31833521 0.32058493]
0.00012283706806093513
[0.34035088 0.3245614  0.33508772]
0.00012151110643949052
[0.3221831  0.35387324 0.32394366]
0.00044401321471481374
[0.34823848 0.33130081 0.3204607 ]
[0.33310742 0.32429685 0.34259573]
0.0002875514678054009
[0.33491686 0.3365004  0.32858274]
0.00010048632861804918
[0.32472178 0.34697933 0.32829889]
0.000319853087990911
[0.34301977 0.32444578 0.33253445]
8.358422169652872e-05
[0.33459945 0.33408149 0.33131906]
0.00011625236123988945
[0.33769814 0.33631978 0.32598208]
6.171881169909022e-05
[0.34823848 0.33130081 0.3204607 ]
[0.33825406 0.33151084 0.3302351 ]
7

3.4664049061188076e-06
[0.34002361 0.3270366  0.33293979]
9.101158849378568e-05
[0.34823848 0.33130081 0.3204607 ]
[0.33997155 0.33854908 0.32147937]
4.4840518756070175e-05
[0.32944345 0.32136445 0.3491921 ]
0.00047659757776036114
[0.32142857 0.32668067 0.35189076]
0.0006442689075003624
[0.31840448 0.34429671 0.33729881]
0.0005043634011950445
[0.34088457 0.33495146 0.32416397]
2.993968790604064e-05
[0.32069672 0.32479508 0.3545082 ]
0.0007296435876174121
[0.34823848 0.33130081 0.3204607 ]
[0.33463833 0.34004474 0.32531693]
0.00010532959867891011
[0.34199497 0.33340319 0.32460184]
2.2428196836601657e-05
[0.3260915  0.33883434 0.33507416]
0.0002845583194849136
[0.32700838 0.33242977 0.34056185]
0.0003202940873167315
[0.33772136 0.32957217 0.33270647]
9.885158231195128e-05
[0.33740953 0.327427   0.33516346]
0.0001309038000331222
[0.34823848 0.33130081 0.3204607 ]
[0.33801437 0.32168517 0.34030046]
0.0002224675095963095
[0.31814672 0.35057915 0.33127413]
0.0005211650804563668
[0.34335686 0

[0.31626898 0.34577007 0.33796095]
0.0005785216871843125
[0.33667522 0.32898111 0.33434367]
0.0001244476034127815
[0.34823848 0.33130081 0.3204607 ]
[0.33139136 0.32488005 0.34372858]
0.00032402276569142317
[0.32838634 0.32650177 0.3451119 ]
0.0003828063595172197
[0.32552693 0.33099141 0.34348165]
0.00039117153731705093
[0.34427452 0.32189482 0.33383066]
0.00010784997923678793
[0.34566787 0.33393502 0.32039711]
4.9900858132003216e-06
[0.32872117 0.33081761 0.34046122]
0.0002921813280895766
[0.34823848 0.33130081 0.3204607 ]
[0.32712082 0.33419023 0.33868895]
0.00029434671850233886
[0.34185304 0.35782748 0.30031949]
0.0004336468072463982
[0.35264901 0.32284768 0.32450331]
4.058398312579083e-05
[0.35928144 0.34431138 0.29640719]
0.00034036225951637544
[0.33702532 0.3164557  0.34651899]
0.0003856292609045031
[0.34064969 0.35206321 0.30728709]
0.00024779273635893573
[0.34823848 0.33130081 0.3204607 ]
[0.3342723  0.32910798 0.33661972]
0.00017265118388937794
[0.346 0.316 0.338]
0.0002090770

[0.35324232 0.33788396 0.30887372]
7.845544927626492e-05
[0.33432709 0.32793867 0.33773424]
0.00018854443422758885
[0.34050881 0.3334638  0.3260274 ]
3.5431386480800305e-05
[0.32987805 0.35426829 0.31585366]
0.00032510911777061045
[0.31674959 0.35157546 0.33167496]
0.0005718790329940509
[0.34056122 0.33886054 0.32057823]
4.271935161432788e-05
[0.34823848 0.33130081 0.3204607 ]
[0.34463277 0.36440678 0.29096045]
0.0007550889934470335
[0.33449477 0.33449477 0.33101045]
0.00011572366251432654
[0.3412527  0.31317495 0.34557235]
0.0003820229062385252
[0.30519481 0.33333333 0.36147186]
0.0013292199721616556
[0.34844869 0.32696897 0.32458234]
1.3726673681650079e-05
[0.36565978 0.31319555 0.32114467]
0.00023368747735399905
[0.34823848 0.33130081 0.3204607 ]
[0.32379395 0.3524121  0.32379395]
0.00038975189677975135
[0.3375481  0.32930181 0.33315008]
0.00010477656508235604
[0.31906977 0.35069767 0.33023256]
0.0004936576706177608
[0.31701445 0.34189406 0.34109149]
0.0005693035885722338
[0.3418745

[0.33982301 0.32430225 0.33587474]
0.00013493199247194097
[0.3426862  0.31565917 0.34165463]
0.00027539382007749064
[0.33552902 0.33539179 0.32907918]
9.393432652685643e-05
[0.33173864 0.33878887 0.32947249]
0.00015227981051518577
[0.34823848 0.33130081 0.3204607 ]
[0.33409402 0.32405294 0.34185304]
0.0002661703682386279
[0.34287043 0.3326355  0.32449407]
1.7400816336816378e-05
[0.31756757 0.33622909 0.34620335]
0.0006112910768543644
[0.32850447 0.34000852 0.331487  ]
0.00021878030328703317
[0.32779661 0.33966102 0.33254237]
0.00023652380644664296
[0.32593493 0.34204926 0.33201581]
0.0002786145114271261
[0.34823848 0.33130081 0.3204607 ]
[0.327654   0.33813893 0.33420708]
0.00024642369196629514
[0.33596392 0.30552424 0.35851184]
0.0008498981141867275
[0.34064786 0.33960293 0.31974922]
4.6793740132460816e-05
[0.34823848 0.33130081 0.3204607 ]
[0.34411277 0.32421227 0.33167496]
7.352717450537253e-05
[0.33556842 0.33587321 0.32855837]
9.179253329740803e-05
[0.32432432 0.33402633 0.3416493

## Functions for Gibbs sampling of variables

In [33]:
def sample_edge_influence(edge_influence, lambda_parameter, theta, indx_sel, K):
    n_spots = len(indx_sel)
    X = K
    for spot in range(n_spots):
        p = theta[spot]
        index = 1
        n_neighbours = len(indx_sel[spot][1:])
        for index in range(n_neighbours):
            neighbour = indx_sel[spot][index+1]
            q = theta[neighbour]
            edge_influence[spot][index] = np.random.exponential(lambda_parameter + Bhatt_dist(p, q, X))

In [34]:
def sample_lambda(E, edge_influence, lambda_a = 0.01, lambda_b = 0.01):
    sum_edge_influences = 0
    for edge in edge_influence:
        sum_edge_influences += sum(edge)
    shape = lambda_a + E
    rate = 1 / (lambda_b + sum_edge_influences)
    lambda_parameter = np.random.gamma(shape, rate, 1)
    return lambda_parameter

In [35]:
lambda_parameter = sample_lambda(E, edge_influence)

In [36]:
def draw_new_factor(umi_factors, ids, spot, w, dt, wt, theta, K, beta = 0.1):
    factor0 = umi_factors[spot][w]
    gene_id = ids[spot][w]
    dt[spot, factor0] -= 1
    wt[factor0, gene_id] -= 1
    left = theta[spot,]
    right = (wt[:,gene_id] + beta) / (wt.sum(axis=0)[gene_id] + K * beta)
    pvals = left*right
    pvals = pvals / pvals.sum()
    factor1 = np.random.choice(K, p = pvals)
    dt[spot, factor1] +=  1
    wt[factor1, gene_id] += 1
    umi_factors[spot][w] = factor1

In [37]:
draw_new_factor(umi_factors, ids, 0, 0, dt, wt, theta, 3)

## Metropolis-Hastings sampling of theta

In [38]:
def metr_hast(theta, indx_sel, edge_influence, lambda_parameter, iterations = 1000):
    
    # Prepare proposal distribution    
    dist_g = lambda x: np.random.dirichlet(x)

    n_spots, X = theta.shape
    
    # Metropolis-Hastings sampling
    for it in range(iterations):
        for spot in range(n_spots):
            log_a_prob0 = 0 # Cannot use from last Gibbs iteration, because there will be new edge_infl and lambdas.
            log_a_prob1 = 0
            p0 = theta[spot]
            p1 = dist_g(np.sum(theta, axis=0)) 
            neighbours = indx_sel[spot][1:]
            index = 0
            for neighbour in neighbours:
                q = theta[neighbour]
                log_edge_potential0 = - Bhatt_dist(p0,q,X) * edge_influence[spot][index] # Double check
                log_edge_potential1 = - Bhatt_dist(p1,q,X) * edge_influence[spot][index]
                log_a_prob0 += log_edge_potential0
                log_a_prob1 += log_edge_potential1
                index += 1
            a_prob = min(1, np.exp(log_a_prob1 - log_a_prob0)) 
#            print(a_prob) # This is really large. Maybe that's normal because everything is better than random?
            if np.random.random() < a_prob:
                p0 = p1
            theta[spot] = p0

In [39]:
%%time
# 10 iterations
metr_hast(theta, indx_sel, edge_influence, lambda_parameter, iterations = 10)

CPU times: user 3.16 s, sys: 20.5 ms, total: 3.18 s
Wall time: 3.19 s


In [40]:
%%time
# 1000 iterations
metr_hast(theta, indx_sel, edge_influence, lambda_parameter)

CPU times: user 5min 12s, sys: 439 ms, total: 5min 12s
Wall time: 5min 12s


## Gibbs sampling

In [41]:
def Gibbs(umi_factors, ids, dt, wt, theta, K, iterations, edge_influence, lambda_parameter, indx_sel, E, beta = 0.1):
    for it in range(iterations):
        for spot in range(len(umi_factors)):
            for w in range(len(umi_factors[spot])):
                draw_new_factor(umi_factors, ids, spot, w, dt, wt, theta, K)
        sample_edge_influence(edge_influence, lambda_parameter, theta, indx_sel, K)
        lambda_parameter = sample_lambda(E, edge_influence, lambda_a = 0.01, lambda_b = 0.01)
        metr_hast(theta, indx_sel, edge_influence, lambda_parameter)

In [42]:
%%time
Gibbs(umi_factors, ids, dt, wt, theta, 3, 1, edge_influence, lambda_parameter, indx_sel, E)

CPU times: user 9min 51s, sys: 957 ms, total: 9min 52s
Wall time: 9min 52s
